# 05. K-Means 클러스터링

비지도 학습의 대표적인 알고리즘인 K-Means를 구현합니다.

## 학습 목표
- K-Means 알고리즘 이해
- K-Means++ 초기화 이해
- 클러스터 수 선택 (Elbow Method)

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)

## 1. 샘플 데이터 생성

In [ ]:
# 3개의 클러스터 생성
n_samples = 300

cluster1 = torch.randn(n_samples // 3, 2) * 0.5 + torch.tensor([0.0, 0.0])
cluster2 = torch.randn(n_samples // 3, 2) * 0.5 + torch.tensor([3.0, 3.0])
cluster3 = torch.randn(n_samples // 3, 2) * 0.5 + torch.tensor([0.0, 3.0])

X = torch.cat([cluster1, cluster2, cluster3], dim=0)
true_labels = torch.cat([torch.zeros(n_samples // 3), 
                         torch.ones(n_samples // 3), 
                         torch.full((n_samples // 3,), 2)])

# 시각화
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=true_labels, cmap='viridis', alpha=0.7)
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Sample Data (3 Clusters)')
plt.colorbar(label='True Cluster')
plt.show()

## 2. K-Means 알고리즘

1. k개의 중심 초기화
2. 각 점을 가장 가까운 중심에 할당
3. 각 클러스터의 평균으로 중심 업데이트
4. 수렴할 때까지 2-3 반복

In [ ]:
from mlfs.classical.clustering import KMeans, KMeansPP
from mlfs.utils.viz import plot_clusters

In [ ]:
# K-Means 클러스터링
kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(X)

# 결과 시각화
plot_clusters(X, labels, centers=kmeans.centers, title='K-Means Clustering')

## 3. K-Means++ 초기화

더 좋은 초기화로 수렴을 개선합니다.

In [ ]:
# K-Means++ 클러스터링
kmeans_pp = KMeansPP(n_clusters=3, random_state=42)
labels_pp = kmeans_pp.fit_predict(X)

# 결과 시각화
plot_clusters(X, labels_pp, centers=kmeans_pp.centers, title='K-Means++ Clustering')

## 4. Elbow Method

최적의 클러스터 수를 찾습니다.

In [ ]:
# 여러 k 값에 대해 inertia 계산
inertias = []
k_range = range(1, 10)

for k in k_range:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X)
    
    # Inertia: 각 점과 중심 간 거리 제곱합
    distances = torch.cdist(X, km.centers)
    min_distances = distances.min(dim=1).values
    inertia = (min_distances ** 2).sum().item()
    inertias.append(inertia)

# Elbow 시각화
plt.figure(figsize=(8, 5))
plt.plot(list(k_range), inertias, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.axvline(x=3, color='r', linestyle='--', label='Optimal k=3')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. MNIST 클러스터링

In [ ]:
from mlfs.utils.data import load_mnist_subset

# 작은 서브셋 로드
X_mnist, y_mnist = load_mnist_subset(n_samples=1000, flatten=True)
print(f"MNIST subset: {X_mnist.shape}")

In [ ]:
# 10개 클러스터로 K-Means
kmeans_mnist = KMeansPP(n_clusters=10, max_iters=100)
cluster_labels = kmeans_mnist.fit_predict(X_mnist)

# 클러스터별 실제 레이블 분포 확인
print("Cluster -> True Label Distribution:")
for c in range(10):
    mask = cluster_labels == c
    if mask.sum() > 0:
        true_labels_in_cluster = y_mnist[mask]
        most_common = torch.mode(true_labels_in_cluster).values.item()
        count = (true_labels_in_cluster == most_common).sum().item()
        total = mask.sum().item()
        print(f"  Cluster {c}: Most common digit = {most_common} ({count}/{total} = {count/total:.1%})")

## 요약

1. **K-Means**: 반복적 할당 및 업데이트로 클러스터링
2. **K-Means++**: 똑똑한 초기화로 더 나은 결과
3. **Elbow Method**: 최적 k 선택
4. **한계**: 구형 클러스터 가정, k를 미리 지정해야 함

다음: 비선형 클러스터링을 위한 **스펙트럴 클러스터링**